In [22]:
from eda_module import Preprocessing
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier




In [ ]:
#initialze classes, constansts and variables
COLORS = sns.color_palette('bright')[0:5]
DATASET = pd.read_csv('heart_failure_clinical_records_dataset.csv')
preprocessor = Preprocessing(DATASET)

#check for missing value and repair it if present
missing_data_removed_Dataset = preprocessor.check_missing_value()

#show descriptive statistics of the dataset, including mean median skewness and kurtosis
preprocessor.descriptives(missing_data_removed_Dataset)

# exploratory data analysis using visualizations
#countplot distribution of death event
sns.countplot(x='DEATH_EVENT', data=missing_data_removed_Dataset)
plt.show()

#boxplot distribution Death Event in respect to diabetes and age
sns.boxplot(x='diabetes', y='age', hue='DEATH_EVENT', data=missing_data_removed_Dataset)
plt.show()

sns.catplot(x='DEATH_EVENT', y='ejection_fraction', data=missing_data_removed_Dataset, kind='box')
plt.show()

plt.pie(missing_data_removed_Dataset['diabetes'].value_counts(), colors=COLORS, labels=['No Diabetes', 'Diabetes'], autopct='%1.1f%%', shadow=True, startangle=90)
plt.show()



In [24]:
#Classification I
#split the dataset into training and test set
X_Train, X_Test, Y_Train, Y_Test = train_test_split(missing_data_removed_Dataset.drop('DEATH_EVENT', axis=1), missing_data_removed_Dataset['DEATH_EVENT'], test_size=0.2, random_state=42)

#Fitting Maching Learning Models
## Support Vector Machine
Support_vector_machine = SVC(C = .1, kernel='linear', random_state=42)
Support_vector_machine.fit(X_Train, Y_Train)

## Random Forest Classifier
random_forest_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
random_forest_classifier.fit(X_Train, Y_Train)

## Naive Bayes
Naive_Bayes = GaussianNB()
Naive_Bayes.fit(X_Train, Y_Train)

## Logistic Regression
Logistic_Regression = LogisticRegression(random_state=42)
Logistic_Regression.fit(X_Train, Y_Train)


##Knearst Neighbors
Knn_model = KNeighborsClassifier(n_neighbors=5)
Knn_model.fit(X_Train, Y_Train)

## Multi Layer Perceptron
Multi_layer_perceptron = MLPClassifier(hidden_layer_sizes=(100, 100, 100), max_iter=1000)
Multi_layer_perceptron.fit(X_Train, Y_Train)

print('All Models have been fitted successfully')


All Models have been fitted successfully
